#Phase 2: Modeling

In [1]:
#from sklearn.model_selection import GridSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import tensorflow as tf
import sklearn as skl

**Train, Test, Validation**

In [2]:
#split validation set by 20%
train_gen = ImageDataGenerator(rescale = 1./255, validation_split=0.20)
test_gen = ImageDataGenerator(rescale = 1./255)
#Image generator helps creating image aumentation to increase the amount of data we have
#Will implement various rotations and flips to the images to distort

In [3]:
def get_train_set(train_gen, input_size):
  train_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                         class_mode='categorical',
                                         target_size= input_size,
                                         color_mode = 'grayscale',
                                         batch_size= 32, 
                                         shuffle = True, 
                                         subset ='training')
  return train_set

In [4]:
def get_validation_set(train_gen, input_size):
  validation_set = train_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/train",
                                          target_size= input_size,
                                          color_mode = 'grayscale',
                                          class_mode='categorical',
                                          batch_size= 32, 
                                          shuffle = True, 
                                          subset ='validation')
  return validation_set

In [5]:
def get_test_set(test_gen, input_size):
  test_set = test_gen.flow_from_directory("/content/drive/MyDrive/DS 2 dataset/test",
                                         target_size=input_size, 
                                         color_mode = 'grayscale',
                                         class_mode='categorical',
                                         batch_size= 1,
                                         shuffle = True)
  return test_set

#Modeling & Hyper-parameter Tuning

In [6]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 4.9MB/s 


In [7]:
from keras.applications import mobilenet_v2 # least parameters (3 mil)
from keras.applications import ResNet101 # (25 mil)
from keras.applications import xception #most parameters (44 mil)
import kerastuner as kt
from kerastuner.tuners import RandomSearch

In [8]:
def build_mobileNet(hp):
  model = mobilenet_v2.MobileNetV2(input_shape=(224,224,1),
      weights = None,
      classes = 196
  )
  
  model.compile(optimizer= tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2])),
                loss = 'categorical_crossentropy',
                metrics = 'accuracy')
  return model 

In [9]:
def build_resNet(hp):
  model = ResNet101(weights = None, classes = 196)

  model.compile(optimizer= tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2])),
                loss = 'categorical_crossentropy',
                metrics = 'accuracy')
    
  return model

In [10]:
tuner_1 = RandomSearch(hypermodel=build_mobileNet, objective='val_accuracy', max_trials= 1, seed = 25)
tuner_2 = RandomSearch(hypermodel=build_resNet,objective='val_accuracy', max_trials= 1, seed = 25)

In [11]:
tuner_1.search_space_summary()

Search space summary
Default search space size: 1
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01], 'ordered': True}


In [12]:
tuner_2.search_space_summary()

Search space summary
Default search space size: 1
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01], 'ordered': True}


In [ ]:
tuner_1.search(get_train_set(train_gen=train_gen, input_size = (224,224)), epochs = 50, validation_data = get_validation_set(train_gen=train_gen, input_size= (224,224)))

Found 6755 images belonging to 196 classes.
Found 1585 images belonging to 196 classes.

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.01              |?                 

Epoch 1/50
212/212 [==============================] - 2417s 11s/step - loss: 6.2820 - accuracy: 0.0065 - val_loss: 5.2678 - val_accuracy: 0.0107
Epoch 2/50
212/212 [==============================] - 47s 221ms/step - loss: 5.2598 - accuracy: 0.0127 - val_loss: 5.2876 - val_accuracy: 0.0107
Epoch 3/50
212/212 [==============================] - 46s 216ms/step - loss: 5.2393 - accuracy: 0.0139 - val_loss: 5.2726 - val_accuracy: 0.0107
Epoch 4/50
212/212 [==============================] - 46s 217ms/step - loss: 5.2195 - accuracy: 0.0150 - val_loss: 5.2756 - val_accuracy: 0.0107
Epoch 5/50
212/212 [==============================] - 46s 216ms/step - loss: 5.1841 - accuracy: 0.0158 - val_loss: 5.2847 - val_accuracy: 0.0095
Epoch 6/50
212/212 [=========================

In [ ]:
tuner_2.search(get_train_set(train_gen=train_gen, input_size = (224,224,3)), epochs = 50, validation_data = get_validation_set(train_gen=train_gen, input_size= (224,224,3)))